# BigQuery Integration Example

This notebook demonstrates how to use BigQuery with the `colab_env` environment for data analysis.

## Prerequisites
1. Environment activated: `mamba activate colab_env`
2. Google Cloud authentication completed
3. PROJECT_ID environment variable set

## Setup Instructions
```bash
# In terminal:
export PROJECT_ID="your-project-id"
gcloud auth application-default login
```

## Method 1: Using google-cloud-bigquery Client

In [ ]:
import os
import pandas as pd
from google.cloud import bigquery
import matplotlib.pyplot as plt
import seaborn as sns

# Set up project ID
PROJECT_ID = os.environ.get('PROJECT_ID')
if not PROJECT_ID:
    PROJECT_ID = input("Enter your Google Cloud Project ID: ")

print(f"Using project: {PROJECT_ID}")

In [ ]:
# Initialize BigQuery client
client = bigquery.Client(project=PROJECT_ID)

# Test connection by listing datasets
print("Available datasets:")
try:
    datasets = list(client.list_datasets())
    if datasets:
        for dataset in datasets[:5]:  # Show first 5
            print(f"  - {dataset.dataset_id}")
    else:
        print("  No datasets found in this project")
except Exception as e:
    print(f"Error listing datasets: {e}")

## Example 1: Query Public Dataset

In [ ]:
# Query USA names public dataset
query = """
SELECT 
    name,
    gender,
    SUM(number) as total_count
FROM `bigquery-public-data.usa_names.usa_1910_current`
WHERE year >= 2010
GROUP BY name, gender
ORDER BY total_count DESC
LIMIT 20
"""

# Execute query
print("Executing query...")
query_job = client.query(query)
df_names = query_job.to_dataframe()

print(f"Query returned {len(df_names)} rows")
df_names.head(10)

In [ ]:
# Visualize the results
plt.figure(figsize=(12, 6))

# Top 10 names by gender
top_names = df_names.head(10)
colors = ['lightblue' if gender == 'M' else 'lightpink' for gender in top_names['gender']]

plt.bar(range(len(top_names)), top_names['total_count'], color=colors)
plt.xlabel('Name')
plt.ylabel('Total Count (2010+)')
plt.title('Top 10 Most Popular Names in US (2010-Current)')
plt.xticks(range(len(top_names)), 
           [f"{row['name']} ({row['gender']})" for _, row in top_names.iterrows()], 
           rotation=45)
plt.tight_layout()
plt.show()

## Method 2: Using pandas-gbq

In [ ]:
import pandas_gbq

# Query using pandas-gbq for easier DataFrame integration
query_trends = """
SELECT 
    year,
    gender,
    SUM(number) as total_births
FROM `bigquery-public-data.usa_names.usa_1910_current`
WHERE year >= 1950 AND year <= 2020
GROUP BY year, gender
ORDER BY year, gender
"""

print("Querying birth trends...")
df_trends = pandas_gbq.read_gbq(query_trends, project_id=PROJECT_ID)

print(f"Retrieved {len(df_trends)} rows")
df_trends.head()

In [ ]:
# Analyze and visualize birth trends
plt.figure(figsize=(12, 8))

# Pivot data for easier plotting
df_pivot = df_trends.pivot(index='year', columns='gender', values='total_births')

# Plot trends
plt.subplot(2, 1, 1)
plt.plot(df_pivot.index, df_pivot['M'], label='Male', color='blue', linewidth=2)
plt.plot(df_pivot.index, df_pivot['F'], label='Female', color='pink', linewidth=2)
plt.title('US Birth Trends by Gender (1950-2020)')
plt.xlabel('Year')
plt.ylabel('Total Births')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot ratio
plt.subplot(2, 1, 2)
ratio = df_pivot['M'] / df_pivot['F']
plt.plot(df_pivot.index, ratio, color='green', linewidth=2)
plt.title('Male to Female Birth Ratio')
plt.xlabel('Year')
plt.ylabel('Ratio (M/F)')
plt.grid(True, alpha=0.3)
plt.axhline(y=1.0, color='red', linestyle='--', alpha=0.7, label='Equal ratio')
plt.legend()

plt.tight_layout()
plt.show()

# Summary statistics
print("\nSummary Statistics:")
print(f"Average male births per year: {df_pivot['M'].mean():,.0f}")
print(f"Average female births per year: {df_pivot['F'].mean():,.0f}")
print(f"Average M/F ratio: {ratio.mean():.3f}")

## Method 3: Using BigQuery Magic Commands

In [ ]:
# Load BigQuery magic extension
%load_ext google.cloud.bigquery

In [ ]:
%%bigquery df_states --project $PROJECT_ID
SELECT 
    state,
    SUM(number) as total_births,
    COUNT(DISTINCT name) as unique_names
FROM `bigquery-public-data.usa_names.usa_1910_current`
WHERE year >= 2000
GROUP BY state
ORDER BY total_births DESC
LIMIT 15

In [ ]:
# Analyze state data
print("Top 15 states by births (2000+):")
display(df_states)

# Visualize
plt.figure(figsize=(12, 8))
plt.barh(df_states['state'], df_states['total_births'])
plt.xlabel('Total Births (2000+)')
plt.title('Total Births by State (2000-Current)')
plt.gca().invert_yaxis()  # Highest at top
plt.tight_layout()
plt.show()

# Correlation between population and name diversity
plt.figure(figsize=(10, 6))
plt.scatter(df_states['total_births'], df_states['unique_names'], alpha=0.7)
plt.xlabel('Total Births')
plt.ylabel('Unique Names')
plt.title('Relationship between Population and Name Diversity by State')

# Add state labels for top 5
for i, row in df_states.head(5).iterrows():
    plt.annotate(row['state'], (row['total_births'], row['unique_names']), 
                xytext=(5, 5), textcoords='offset points', fontsize=8)

plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Working with Your Own Data

In [ ]:
# Example: Create a sample dataset and upload to BigQuery
# (Uncomment and modify for your actual use case)

# Create sample financial data
import numpy as np
from datetime import datetime, timedelta

# Generate sample stock price data
np.random.seed(42)
dates = pd.date_range(start='2023-01-01', end='2024-01-01', freq='D')
sample_data = {
    'date': dates,
    'symbol': np.random.choice(['AAPL', 'GOOGL', 'MSFT', 'TSLA'], len(dates)),
    'price': np.random.uniform(100, 300, len(dates)),
    'volume': np.random.randint(1000000, 10000000, len(dates))
}

df_sample = pd.DataFrame(sample_data)
print("Sample financial data:")
print(df_sample.head())

# To upload to BigQuery (uncomment to use):
# dataset_id = 'your_dataset_name'
# table_id = 'sample_stock_data'
# 
# df_sample.to_gbq(
#     f'{dataset_id}.{table_id}',
#     project_id=PROJECT_ID,
#     if_exists='replace',
#     progress_bar=True
# )
# print(f"Data uploaded to {PROJECT_ID}.{dataset_id}.{table_id}")

## Cost Management Tips

In [ ]:
# Check query cost before running expensive queries
expensive_query = """
SELECT *
FROM `bigquery-public-data.usa_names.usa_1910_current`
WHERE year >= 1910
"""

# Get query job config to estimate cost
job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)
query_job = client.query(expensive_query, job_config=job_config)

print(f"This query will process {query_job.total_bytes_processed:,} bytes")
print(f"Estimated cost: ${(query_job.total_bytes_processed / 1024**4) * 5:.4f} USD")
print("(Based on $5 per TB processed)")

## Next Steps

1. **Explore your own datasets**: Replace the public dataset queries with your own data
2. **Set up scheduled queries**: Use BigQuery's scheduled queries for regular data updates
3. **Integrate with other tools**: Combine with Cloud Storage, Sheets, or other GCP services
4. **Monitor costs**: Set up billing alerts and use query cost estimation
5. **Optimize queries**: Use partitioning, clustering, and appropriate WHERE clauses

## Useful Resources

- [BigQuery Documentation](https://cloud.google.com/bigquery/docs)
- [pandas-gbq Documentation](https://pandas-gbq.readthedocs.io/)
- [BigQuery Magic Commands](https://cloud.google.com/bigquery/docs/bigquery-magic)
- [BigQuery Public Datasets](https://cloud.google.com/bigquery/public-data)